# Appendix C, Example 1

## Introduction

In [ ]:
(refer 'sparqler.core)
(assert kabob)

## Query

### Manual SPARQL

The following cell will be stored in the beakerx.query1 variable....

In [ ]:
%%sql
PREFIX franzOption_clauseReorderer: <franz:identity>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX obo: <http://purl.obolibrary.org/obo/>
PREFIX kbio: <http://kabob.ucdenver.edu/bio/>
PREFIX iaohgnc: <http://kabob.ucdenver.edu/iao/hgnc/>


SELECT ?participant 
WHERE {

iaohgnc:HGNC_SEPT2_ICE obo:IAO_0000219 ?gene  . # SEPT2

## gene specific gene or gene product abstraction ?gorgporv
?gene rdfs:subClassOf* ?gorgporv .  
?gorgporv rdf:type kbio:GeneSpecificGorGPorVClass .

## all gene products
?geneproduct rdfs:subClassOf* ?gorgporv.

## get interactions they participate in
?participant_r owl:someValuesFrom ?geneproduct.
?participant_r owl:onProperty obo:RO_0000057 . # RO:has_participant
?interaction rdfs:subClassOf ?participant_r .

## look at only interactions
?interaction rdfs:subClassOf obo:MI_0407 . # interaction

## get all the other participants in those interactions
?interaction rdfs:subClassOf ?participant_r2.
FILTER (?participant_r != ?participant_r2) . # insure different
?participant_r2 owl:onProperty obo:RO_0000057 . # RO:has_participant
?participant_r2 owl:someValuesFrom ?participant.
}

In [ ]:
%%javascript
beakerx.query1 = beakerx.getCodeCells('query1')[0].code

### SPARQLer

#### Step 1: Convert to clojure and define constants

In [ ]:
(in-ns 'sparqler.core)

(def MI:interaction [:obo :MI_0407])

(defquery septin2-participants []
  
  (select :participant)
  
  (where 
    ;iaohgnc:HGNC_SEPT2_ICE obo:IAO_0000219 ?gene  .
    [:iaohgnc :HGNC_SEPT2_ICE] iao:denotes :gene \.

    ;## gene specific gene or gene product abstraction ?gorgporv
    ;?gene rdfs:subClassOf* ?gorgporv .  
    ;?gorgporv rdf:type kbio:GeneSpecificGorGPorVClass .
    (gene-or-gene-products :gene :gorgporv)
    
    ;## all gene products
    ;?geneproduct rdfs:subClassOf* ?gorgporv.
    :geneproduct rdfs:subClassOf* :gorgporv \.

    ;## get interactions they participate in
    :participant_r owl:someValuesFrom :geneproduct \.
    :participant_r owl:onProperty RO:has_participant \. ;# RO:has_participant
    :interaction rdfs:subClassOf :participant_r \.

    ;## look at only interactions
    :interaction rdfs:subClassOf MI:interaction \. ;# interaction

    ;## get all the other participants in those interactions
    :interaction rdfs:subClassOf :participant_r2 \.
    (filter :participant_r != :participant_r2) \. ;# insure different
    :participant_r2 owl:onProperty RO:has_participant \. ;# RO:has_participant
    :participant_r2 owl:someValuesFrom :participant \.
    )
  )


In [ ]:
(#'sparqler.core/septin2-participants)

In [ ]:
(beakerx :set "query2" (#'sparqler.core/septin2-participants))

In [ ]:
%%html
<pre id="display"></pre>

In [ ]:
%%javascript
document.getElementById("display").innerHTML = "";

var one = beakerx.query1,
    other = beakerx.query2,
    color = 'blue',
    span = null;

var diff = JsDiff.diffChars(one, other),
    display = document.getElementById('display'),
    fragment = document.createDocumentFragment();

diff.forEach(function(part){
  // green for additions, red for deletions
  // grey for common parts
  color = part.added ? 'green' :
    part.removed ? 'red' : 'grey';
  span = document.createElement('span');
  span.style.color = color;
  span.appendChild(document
    .createTextNode(part.value));
  fragment.appendChild(span);
});

display.appendChild(fragment);

In [ ]:
(kabob-query (angiogenesis))